In [ ]:
pip install torch

In [2]:
# Import libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


In [3]:
class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads
        
        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size needs to be divisible by heads"
        
        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)
    
    def forward(self, values, keys, query, mask):
        N = query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]
        
        # Split the embedding into self.heads different pieces
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = query.reshape(N, query_len, self.heads, self.head_dim)
        
        values = self.values(values)
        keys = self.keys(keys)
        queries = self.queries(queries)
        
        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))
        
        attention = torch.nn.functional.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)
        
        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )
        
        out = self.fc_out(out)
        return out


In [4]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)
        
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size),
        )
        
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)
        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out


In [5]:
class Encoder(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        embed_size,
        num_layers,
        heads,
        device,
        forward_expansion,
        dropout,
        max_length,
    ):
        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.device = device
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)
        
        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_size,
                    heads,
                    dropout=dropout,
                    forward_expansion=forward_expansion,
                )
                for _ in range(num_layers)
            ]
        )
        
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        out = self.dropout(self.word_embedding(x) + self.position_embedding(positions))
        
        for layer in self.layers:
            out = layer(out, out, out, mask)
        
        return out


In [6]:
class Transformer(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        embed_size,
        num_layers,
        heads,
        device,
        forward_expansion,
        dropout,
        max_length,
        trg_vocab_size,
    ):
        super(Transformer, self).__init__()
        self.encoder = Encoder(
            src_vocab_size,
            embed_size,
            num_layers,
            heads,
            device,
            forward_expansion,
            dropout,
            max_length,
        )
        
        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, src, mask):
        enc_src = self.encoder(src, mask)
        out = self.fc_out(enc_src)
        return out


In [7]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define model parameters
src_vocab_size = 1000
trg_vocab_size = 1000
src_pad_idx = 0
trg_pad_idx = 0
src_sentence_length = 10
trg_sentence_length = 10
d_model = 256
heads = 8
num_layers = 3
forward_expansion = 4
dropout = 0.1

# Create random input and mask
src_input = torch.randint(0, src_vocab_size, (src_sentence_length,)).unsqueeze(0)
mask = (src_input != src_pad_idx).unsqueeze(1).unsqueeze(2)

# Instantiate transformer model
model = Transformer(
    src_vocab_size,
    d_model,
    num_layers,
    heads,
    device,
    forward_expansion,
    dropout,
    src_sentence_length,
    trg_vocab_size,
).to(device)

# Forward pass
output = model(src_input.to(device), mask.to(device))
print(output.shape)  # Output shape (1, trg_sentence_length, trg_vocab_size)
print(output)

torch.Size([1, 10, 1000])
tensor([[[-0.7815, -0.3383,  0.1105,  ..., -0.6749, -0.3067,  0.6858],
         [-0.0656, -0.0026,  0.0956,  ...,  0.5279,  0.4444,  0.1174],
         [ 0.3272, -0.5715,  0.4191,  ...,  0.4235,  0.7578, -0.4480],
         ...,
         [ 0.8900, -0.7988,  0.5660,  ..., -0.5092,  0.4008,  0.4073],
         [-0.7958,  1.2916,  1.1668,  ..., -0.1621,  0.1391, -0.2790],
         [ 1.1782, -0.0309,  0.4312,  ..., -0.2797,  0.2077,  0.3782]]],
       grad_fn=<ViewBackward0>)
